<a href="https://colab.research.google.com/github/sljm12/machine_learning_notebooks/blob/master/Zeroshot_learning_using_BART_and_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is to try Zeroshot learning on News Articles Headings.

The news headings are from IPTC Media Topics https://iptc.org/standards/media-topics/

Baseline code from https://joeddav.github.io/blog/2020/05/29/ZSL.html

In [38]:
!nvidia-smi

Mon Nov 23 03:01:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    28W /  70W |   4205MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!pip -qq install transformers

     |████████████████████████████████| 1.3MB 8.5MB/s 
     |████████████████████████████████| 1.1MB 14.2MB/s 
     |████████████████████████████████| 2.9MB 42.1MB/s 
     |████████████████████████████████| 890kB 50.8MB/s 


# Data Prep

In [2]:
!wget https://www.iptc.org/std/NewsCodes/IPTC-MediaTopic-NewsCodes.xlsx

--2020-11-23 02:34:04--  https://www.iptc.org/std/NewsCodes/IPTC-MediaTopic-NewsCodes.xlsx
Resolving www.iptc.org (www.iptc.org)... 194.232.153.3
Connecting to www.iptc.org (www.iptc.org)|194.232.153.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 628997 (614K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘IPTC-MediaTopic-NewsCodes.xlsx’

IPTC-MediaTopic-New 100%[===================>] 614.25K   764KB/s    in 0.8s    

2020-11-23 02:34:06 (764 KB/s) - ‘IPTC-MediaTopic-NewsCodes.xlsx’ saved [628997/628997]



In [3]:
import pandas as pd
import torch
df=pd.read_excel("IPTC-MediaTopic-NewsCodes.xlsx", skiprows=1)

In [4]:
df.head()

,NewsCode-URI,NewsCode-QCode (flat),Level1/NewsCode,Level2/NewsCode,Level3/NewsCode,Level4/NewsCode,Level5/NewsCode,Level6/NewsCode,RetiredDate,Name (ar),Definition (ar),Name (de),Definition (de),Name (dk),Definition (dk),Name (en-GB),Definition (en-GB),Name (es),Definition (es),Name (fr),Definition (fr),Name (no),Definition (no),Name (pt-PT),Definition (pt-PT),Name (pt-BR),Definition (pt-BR),Name (se),Definition (se),Name (zh-Hans),Definition (zh-Hans),SubjectCode mapping,Wikidata mapping
0,http://cv.iptc.org/newscodes/mediatopic/01000000,medtop:01000000,medtop:01000000,NaN,NaN,NaN,NaN,NaN,NaN,فنون، ثقافة وترفيه,NaN,"Kultur, Kunst, Unterhaltung und Medien","Alle Formen von Kunst, Unterhaltung, Kulturerb...","kunst, kultur, underholdning og medier","Kunst, kultur, underholdning og medier.","arts, culture, entertainment and media","All forms of arts, entertainment, cultural her...","Artes, cultura, entretenimiento y medios","Todas las formas de arte, entretenimiento cult...","Arts, culture, divertissement et médias","Toutes les formes d'arts, de divertissement, d...",kultur og underholdning,NaN,"artes, cultura, entretenimento e média","Todas as formas de arte, entretenimento, heran...","artes, cultura, entretenimento e mídia","Todas as formas de arte, entretenimento, heran...","Konst, kultur och nöje",Ämnen som rör utveckling och förfining av det ...,艺术、文化、娱乐和媒体,各种形式的艺术、娱乐、文化遗产和媒体,http://cv.iptc.org/newscodes/subjectcode/01000000,NaN
1,http://cv.iptc.org/newscodes/mediatopic/20000002,medtop:20000002,NaN,medtop:20000002,NaN,NaN,NaN,NaN,NaN,فنون وتسلية,NaN,Kunst und Unterhaltung,Alle Formen von Kunst und Unterhaltung,kunst og underholdning,Alle former for kunst og underholdning.,arts and entertainment,All forms of arts and entertainment,Arte y entretenimiento,Todas las formas de arte y de entretenimiento,Arts et divertissement,Toutes les formes d'arts et de divertissement,kunst og underholdning,NaN,arte e entretenimento,Todas as formas de arte e entretenimento.,arte e entretenimento,Todas as formas de arte e entretenimento.,Konst och underhållning,Alla former av konst och underhållning som int...,艺术与娱乐,各种形式的艺术和娱乐,NaN,https://www.wikidata.org/entity/Q2018526
2,http://cv.iptc.org/newscodes/mediatopic/20000003,medtop:20000003,NaN,NaN,medtop:20000003,NaN,NaN,NaN,NaN,تحريك,NaN,Animation,"Erzählung in Form von Computergrafik, als Lang...",animation,Historier fortalt gennem animerede tegninger e...,animation,Stories told through animated drawings in eith...,Animación,Historias contadas a través de dibujos animado...,Dessin animé,Histoires racontées à travers des dessins anim...,animasjonsfilm,NaN,animação,Técnica que permite dar a desenhos ou bonecos ...,animação,"Animação, incluindo longas e curtas-metragens,...",Animation,"Rörlig bild skapad av en serie teckningar, ler...",动画,长短篇的动画故事,http://cv.iptc.org/newscodes/subjectcode/01025000,https://www.wikidata.org/entity/Q11425
3,http://cv.iptc.org/newscodes/mediatopic/20001135,medtop:20001135,NaN,NaN,medtop:20001135,NaN,NaN,NaN,NaN,NaN,NaN,Kunstausstellung,Zeitlich begrenzte Präsentation von Kunst in e...,udstilling,Midlertidig præsentation af kunst eller særemn...,art exhibition,"Temporary presentation of art in museums, art ...",Exposición de arte,"Presentación temporal de arte en museos, salas...",Exposition artistique,"Présentation temporaire d'art dans des musées,...",kunstutstilling,NaN,exposição artística,"Apresentação temporária de arte em museus, sal...",exposição artística,"Apresentação temporária de arte em museus, sal...",Konstutställning,Ett tillfälle då en eller flera konstverk visa...,艺术展览,在博物馆、艺术大厅或画廊举办的短期艺术品展出,NaN,https://www.wikidata.org/entity/Q667276
4,http://cv.iptc.org/newscodes/mediatopic/20000004,medtop:20000004,NaN,NaN,medtop:20000004,NaN,NaN,NaN,NaN,كرتون,NaN,Comic,"Zeichnungen, wie Karikaturen und Comics, die o...",tegneserie,"Tegninger, både som underholding og redaktione...",cartoon,"Drawings, such as editorial cartoons and comic...",Caricaturas,"Dibujos, como 

In [5]:
# Get the 2nd level headings because they are more detailed that the first level headings
df_top_level = df[df["Level2/NewsCode"].notnull()]

In [6]:
df_top_level.size

3597

In [7]:
language = "zh-Hans"
#language = "en-GB"
topics = [i for i in df_top_level["Name (%s)"% (language)]]

In [8]:
topics[0:5]

['艺术与娱乐', '文化', '大众传媒', '恐怖活动', '武装冲突']

# Zeroshot learning portion

Use facebook/bart-large-mnli for english. https://huggingface.co/facebook/bart-large-mnli

Use xlm-roberta-large-xnli for multilingual analysis.
For the list of languages look https://huggingface.co/joeddav/xlm-roberta-large-xnli

In [9]:
model_name = 'facebook/bart-large-mnli' #@param ['facebook/bart-large-mnli', "joeddav/xlm-roberta-large-xnli"] {allow-input: true}

In [29]:
from transformers import BartForSequenceClassification, BartTokenizer, AutoModelForSequenceClassification, AutoTokenizer
#tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-mnli')
#model = BartForSequenceClassification.from_pretrained('facebook/bart-large-mnli')


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# pose sequence as a NLI premise and label (politics) as a hypothesis
premise = 'Who are you voting for in 2020?'
hypothesis = 'This text is about politics.'

if torch.cuda.current_device >=0:
    model.to('cuda')

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BartForSequenceClassification(
  (model): BartModel(
    (shared): Embedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 1024, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 1024, padding_idx=1)
      (layers): ModuleList(
        (0): EncoderLayer(
          (self_attn): Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)

In [66]:

def predict(tokenizer, model, premise, hypothesis_sentence, category):
  hypothesis = hypothesis_sentence % (category)

  input_ids = tokenizer.encode(premise, hypothesis, return_tensors='pt')
  if torch.cuda.current_device() >=0:
    input_ids=input_ids.cuda()

  logits = model(input_ids)[0]

  #print(logits)
  entail_contradiction_logits = logits[:,[0,2]]
  #print(entail_contradiction_logits)
  probs = entail_contradiction_logits.softmax(dim=1)
  true_prob = probs[:,1].item() * 100
  #print(f'Probability that the label is true: {true_prob:0.2f}%')
  return true_prob

In [67]:
hypothesis_sentence = {
    "en-GB": "This article is about %s",
    "zh-Hans": "这篇文章是关于%s"
}

In [68]:
sentence = hypothesis_sentence[language]

Just some notes on the using GPU, without GPU it took 2mins with GPU it took 4s

In [69]:
import datetime
start_time=datetime.datetime.now()
premise = "《研究報告》電動自駕飆速 AI基金樂歪"
#hypothesis_sentence = "This article is about %s"

results = []
for i in topics:
  prob = predict(tokenizer, model, premise, sentence, i)
  results.append({
      "category":i,
      "prob":prob
  })
end_time=datetime.datetime.now()
print(end_time-start_time)

0:00:04.224723


In [13]:
result_frame = pd.DataFrame(data=results)

In [14]:
result_frame.sort_values("prob",ascending=False)

,category,prob
81,科学研究,99.933004
80,科研机构,99.728715
79,自然科学,99.623114
77,生物医学科学,99.014181
84,技术和工程,98.902285
...,...,...
14,法律,0.016649
62,政府,0.015955
91,人类,0.014828
63,政府政策,0.014573


# Original Code

In [ ]:
# run through model pre-trained on MNLI
input_ids = tokenizer.encode(premise, hypothesis, return_tensors='pt')
print(input_ids)
logits = model(input_ids)[0]

print(logits)


tensor([[    0, 12375,    32,    47,  3434,    13,    11,  2760,   116,     2,
             2,   713,  2788,    16,    59,  2302,     4,     2]])
tensor([[-2.5443,  0.8770,  1.3904]], grad_fn=<AddmmBackward>)


In [ ]:
# we throw away "neutral" (dim 1) and take the probability of
# "entailment" (2) as the probability of the label being true 
entail_contradiction_logits = logits[:,[0,2]]
print(entail_contradiction_logits)
probs = entail_contradiction_logits.softmax(dim=1)
true_prob = probs[:,1].item() * 100
print(f'Probability that the label is true: {true_prob:0.2f}%')

tensor([[-2.5443,  1.3904]], grad_fn=<IndexBackward>)
Probability that the label is true: 98.08%


In [ ]:
logits[:,[0,1]]

tensor([[-2.5443,  0.8770]], grad_fn=<IndexBackward>)

In [ ]:
probs

tensor([[0.0192, 0.9808]], grad_fn=<SoftmaxBackward>)